## In this notebook i will create and train the first prediction model

In [ ]:
#this code reads the dataframe from the saved csv
import pandas as pd
df = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\modified_btc_data.csv",
    index_col=0,
    parse_dates=True
)
print(df)


In [ ]:
#This code adds a column for the next hour price ,
#  in order for us to do the prediction calculation
df["next_hour"] = df["close"].shift(-1)

In [ ]:
df

In [ ]:
# This code checks if the price went up from a ceratin hour to its next hour
# and the  result type was converted to int for our ML model.
df["Target"] = (df["next_hour"] > df["close"]).astype(int)

In [ ]:
#This code removes all the data that came before a certain date 
#(I used a copy of our dataframe just to check that it works) , the code for our df is 
# df=df.loc["2016-01-30":].copy()
dfcopy=df.loc["2016-01-30":].copy()
print(dfcopy)

In [19]:



# n_estimators -? the number of decision tress we want to learn (higher = better accuarcy up to a limit)
# 100 is a low number - need to exepiremnt with it to find the correct number

# Min_samples_split -> helps us protect aginst overfitting (if the build the tree too deeply) 
# - need to exepiremnt with it to find the correct number

#random state -> if we run the same model twice the random numbers that are genreated will be in a predictable
# sequance each time , using this random seed of 1 (if we run the same model twice we will get same results)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,min_samples_split=100,random_state=1)

train = dfcopy.iloc[:-100] #All the rows except the last 100 rows will be put in the training set
test = dfcopy.iloc[:-100] # The last 100 rows will be put in the test set

# list of all the columns we are going to use to predict the target
predictors = ["close","Volume BTC","open","high","low"]

#This is going to train the model to predict the target using the predictor columns
model.fit(train[predictors],train["Target"])



RandomForestClassifier(min_samples_split=100, random_state=1)

In [20]:
#This code will check our model , (when we said the market would go up did it acctualy go up)
from sklearn.metrics import precision_score

#this will generate predictions using our model
preds = model.predict(test[predictors])


In [21]:
# this will convert our predictions into a series fromat (easier to read)
import pandas as pd 
preds = pd.Series(preds,index=test.index) 

In [24]:
#show the prediction array
preds

date
2016-01-30 00:00:00    0
2016-01-30 01:00:00    0
2016-01-30 02:00:00    0
2016-01-30 03:00:00    0
2016-01-30 04:00:00    0
                      ..
2025-03-19 16:00:00    0
2025-03-19 17:00:00    0
2025-03-19 18:00:00    0
2025-03-19 19:00:00    0
2025-03-19 20:00:00    1
Length: 80083, dtype: int32

In [23]:
#Calculation of the precission score
precision_score(test["Target"],preds)

0.7617517256987666

In [ ]:
#This code plots the predictions
combined = pd.concat([test["Target"],preds],axis=1)

In [26]:
#this code will save the model so u can keep working on it later
import os
import joblib

# Ensure the directory exists
os.makedirs('models', exist_ok=True)

# Save the model
joblib.dump(model, 'models/random_forest_model.pkl')



['models/random_forest_model.pkl']

In [ ]:
#Load the model back up 
import joblib

# Load the saved model from file
model = joblib.load('models/random_forest_model.pkl')
